In [1]:
import sqlite3
import random
import networkx as nx
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


conn = sqlite3.connect("db_numprop-4_nestlim-100.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM data")
all_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
formula_idx = column_names.index("formula")
category_idx = column_names.index("category")

In [2]:
# Utilities

def round_prediction(pred, threshold=0.5):
    return (pred > threshold).float()


def binary_to_bitlist(n, total):
    return [int(a) for a in f'{n:0{total}b}']
    
def tree_to_formula(tree):
    """
    Recursively render a nested tuple/tree
    into a string like "NA(q,s)" or "AND(p,OR(q,r))".
    """
    if not isinstance(tree, tuple):
        return str(tree)

    op, *children = tree
    rendered_children = [tree_to_formula(c) for c in children]
    return f"{op}({','.join(rendered_children)})"

In [3]:
def find_allowable_combinations(tree, correct, assignments, x_counter=0):
    p, q, r, s = assignments

    f, *args = tree
    op = f


    # A helper to union results (for alternatives) while threading the counter.
    def union_results(results):
        union_list = []
        max_counter = x_counter  # starting counter
        for res, cnt in results:
            union_list.extend(res)
            max_counter = max(max_counter, cnt)
        return union_list, max_counter

    # A helper to combine two lists of constraint dictionaries.
    def combine(list1, list2):
        result = []
        for d1 in list1:
            for d2 in list2:
                merged = dict(d1)  # Make sure d1 is treated as a dict
                conflict = False
                for key, value in d2.items():
                    if key in merged and merged[key] != value:
                        conflict = True
                        break
                    merged[key] = value
                if not conflict:
                    result.append(merged)
        return result


    # Process operators.
    if op == 'N':
        res, new_counter = find_allowable_combinations(args[0], 1 - correct, assignments, x_counter)
        return res, new_counter

    elif op == 'A':
        if correct == 1:
            left, counter_left = find_allowable_combinations(args[0], 1, assignments, x_counter)
            right, counter_right = find_allowable_combinations(args[1], 1, assignments, counter_left)
            return combine(left, right), counter_right
        else:
            branch1_left, counter1 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch1_right, counter1 = find_allowable_combinations(args[1], 1, assignments, counter1)
            poss1 = combine(branch1_left, branch1_right)

            branch2_left, counter2 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch2_right, counter2 = find_allowable_combinations(args[1], 0, assignments, counter2)
            poss2 = combine(branch2_left, branch2_right)

            branch3_left, counter3 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch3_right, counter3 = find_allowable_combinations(args[1], 0, assignments, counter3)
            poss3 = combine(branch3_left, branch3_right)

            union_list, final_counter = union_results([
                (poss1, counter1), (poss2, counter2), (poss3, counter3)
            ])
            return union_list, final_counter

    elif op == 'O':
        if correct == 1:
            branch1_left, counter1 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch1_right, counter1 = find_allowable_combinations(args[1], 1, assignments, counter1)
            poss1 = combine(branch1_left, branch1_right)

            branch2_left, counter2 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch2_right, counter2 = find_allowable_combinations(args[1], 0, assignments, counter2)
            poss2 = combine(branch2_left, branch2_right)

            branch3_left, counter3 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch3_right, counter3 = find_allowable_combinations(args[1], 1, assignments, counter3)
            poss3 = combine(branch3_left, branch3_right)

            union_list, final_counter = union_results([
                (poss1, counter1), (poss2, counter2), (poss3, counter3)
            ])
            return union_list, final_counter
        else:
            left, counter_left = find_allowable_combinations(args[0], 0, assignments, x_counter)
            right, counter_right = find_allowable_combinations(args[1], 0, assignments, counter_left)
            return combine(left, right), counter_right

    elif op == 'C':
        if correct == 1:
            branch1_left, counter1 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch1_right, counter1 = find_allowable_combinations(args[1], 0, assignments, counter1)
            poss1 = combine(branch1_left, branch1_right)

            branch2_left, counter2 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch2_right, counter2 = find_allowable_combinations(args[1], 1, assignments, counter2)
            poss2 = combine(branch2_left, branch2_right)

            branch3_left, counter3 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch3_right, counter3 = find_allowable_combinations(args[1], 1, assignments, counter3)
            poss3 = combine(branch3_left, branch3_right)

            union_list, final_counter = union_results([
                (poss1, counter1), (poss2, counter2), (poss3, counter3)
            ])
            return union_list, final_counter
        else:
            left, counter_left = find_allowable_combinations(args[0], 1, assignments, x_counter)
            right, counter_right = find_allowable_combinations(args[1], 0, assignments, counter_left)
            return combine(left, right), counter_right

    elif op == 'NC':
        if correct == 1:
            left, counter_left = find_allowable_combinations(args[0], 1, assignments, x_counter)
            right, counter_right = find_allowable_combinations(args[1], 0, assignments, counter_left)
            return combine(left, right), counter_right
        else:
            branch1_left, counter1 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch1_right, counter1 = find_allowable_combinations(args[1], 0, assignments, counter1)
            poss1 = combine(branch1_left, branch1_right)

            branch2_left, counter2 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch2_right, counter2 = find_allowable_combinations(args[1], 1, assignments, counter2)
            poss2 = combine(branch2_left, branch2_right)

            branch3_left, counter3 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch3_right, counter3 = find_allowable_combinations(args[1], 1, assignments, counter3)
            poss3 = combine(branch3_left, branch3_right)

            union_list, final_counter = union_results([
                (poss1, counter1), (poss2, counter2), (poss3, counter3)
            ])
            return union_list, final_counter

    elif op == 'B':
        if correct == 1:
            branch1_left, counter1 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch1_right, counter1 = find_allowable_combinations(args[1], 1, assignments, counter1)
            poss1 = combine(branch1_left, branch1_right)

            branch2_left, counter2 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch2_right, counter2 = find_allowable_combinations(args[1], 0, assignments, counter2)
            poss2 = combine(branch2_left, branch2_right)

            union_list, final_counter = union_results([
                (poss1, counter1), (poss2, counter2)
            ])
            return union_list, final_counter
        else:
            branch1_left, counter1 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch1_right, counter1 = find_allowable_combinations(args[1], 0, assignments, counter1)
            poss1 = combine(branch1_left, branch1_right)

            branch2_left, counter2 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch2_right, counter2 = find_allowable_combinations(args[1], 1, assignments, counter2)
            poss2 = combine(branch2_left, branch2_right)

            union_list, final_counter = union_results([
                (poss1, counter1), (poss2, counter2)
            ])
            return union_list, final_counter

    elif op == 'X':
        if correct == 1:
            branch1_left, counter1 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch1_right, counter1 = find_allowable_combinations(args[1], 0, assignments, counter1)
            poss1 = combine(branch1_left, branch1_right)

            branch2_left, counter2 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch2_right, counter2 = find_allowable_combinations(args[1], 1, assignments, counter2)
            poss2 = combine(branch2_left, branch2_right)

            union_list, final_counter = union_results([
                (poss1, counter1), (poss2, counter2)
            ])
            return union_list, final_counter
        else:
            branch1_left, counter1 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch1_right, counter1 = find_allowable_combinations(args[1], 1, assignments, counter1)
            poss1 = combine(branch1_left, branch1_right)

            branch2_left, counter2 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch2_right, counter2 = find_allowable_combinations(args[1], 0, assignments, counter2)
            poss2 = combine(branch2_left, branch2_right)

            union_list, final_counter = union_results([
                (poss1, counter1), (poss2, counter2)
            ])
            return union_list, final_counter

    elif op == 'NA':
        if correct == 1:
            branch1_left, counter1 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch1_right, counter1 = find_allowable_combinations(args[1], 0, assignments, counter1)
            poss1 = combine(branch1_left, branch1_right)

            branch2_left, counter2 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch2_right, counter2 = find_allowable_combinations(args[1], 1, assignments, counter2)
            poss2 = combine(branch2_left, branch2_right)

            branch3_left, counter3 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch3_right, counter3 = find_allowable_combinations(args[1], 0, assignments, counter3)
            poss3 = combine(branch3_left, branch3_right)

            union_list, final_counter = union_results([
                (poss1, counter1), (poss2, counter2), (poss3, counter3)
            ])
            return union_list, final_counter
        else:
            left, counter_left = find_allowable_combinations(args[0], 1, assignments, x_counter)
            right, counter_right = find_allowable_combinations(args[1], 1, assignments, counter_left)
            return combine(left, right), counter_right

    elif op == 'NOR':
        if correct == 1:
            left, counter_left = find_allowable_combinations(args[0], 0, assignments, x_counter)
            right, counter_right = find_allowable_combinations(args[1], 0, assignments, counter_left)
            return combine(left, right), counter_right
        else:
            branch1_left, counter1 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch1_right, counter1 = find_allowable_combinations(args[1], 0, assignments, counter1)
            poss1 = combine(branch1_left, branch1_right)

            branch2_left, counter2 = find_allowable_combinations(args[0], 0, assignments, x_counter)
            branch2_right, counter2 = find_allowable_combinations(args[1], 1, assignments, counter2)
            poss2 = combine(branch2_left, branch2_right)

            branch3_left, counter3 = find_allowable_combinations(args[0], 1, assignments, x_counter)
            branch3_right, counter3 = find_allowable_combinations(args[1], 1, assignments, counter3)
            poss3 = combine(branch3_left, branch3_right)

            union_list, final_counter = union_results([
                (poss1, counter1), (poss2, counter2), (poss3, counter3)
            ])
            return union_list, final_counter

    elif op in ('p', 'q', 'r', 's'):
        # Check if the predetermined value agrees with the expected
        val = {'p': p, 'q': q, 'r': r, 's': s}[op]
        return ([] if val != correct else [{}]), x_counter

    elif op == 'Z':
        # For an unknown "X" node, assign a unique variable name.
        new_var = f"Z_{x_counter}"
        return ([{new_var: correct}]), x_counter + 1


In [4]:
def extract_grammar_from_data_row(row, columns):
    """
    Build grammar from a row in the 'data' table.
    Each rule returns a nested tuple.
    """
    operator_names = {
        "A": "A",
        "O": "O",
        "C": "C",
        "NC": "NC",
        "B": "B",
        "X": "X",
        "NA": "NA",
        "NOR": "NOR",
        "N": "N",
    }

    grammar = {}
    for op, name in operator_names.items():
        if op in columns and row[columns.index(op)] == 1:
            if op == "N":
                grammar[op] = lambda Z, name=name: (name, Z)
            else:
                grammar[op] = lambda Z, name=name: (name, Z, Z)
    return grammar



def expand_all_X(expr, grammar):
    """
    Recursively finds the leftmost 'X' in a nested tuple structure and replaces it
    with each possible grammar rule or terminal symbol.
    """
    if expr == "Z":
        # Base case: single 'X' to replace
        expansions = []

        for rule in grammar.values():
            expansions.append(rule("Z"))

        for terminal in ["p", "q", "r", "s"]:
            expansions.append(terminal)

        return expansions

    elif isinstance(expr, tuple):
        # Recursive case: traverse the structure to find the leftmost 'X'
        for i, sub in enumerate(expr):
            sub_expansions = expand_all_X(sub, grammar)
            if sub_expansions:
                # Replace the first expandable part and break
                results = []
                for new_sub in sub_expansions:
                    new_expr = list(expr)
                    new_expr[i] = new_sub
                    results.append(tuple(new_expr))
                return results
    return []


def run_derivation_for_row(row_idx, row, columns, current = None):
    """
    Expands from the starting expression 'X' using grammar derived from the row.
    This version performs a single iteration and returns the current expression
    along with the options dictionary. An external function can use the options dict
    to select the next node.

    Returns:
        current (str): The starting expression (or new node if already set).
        current_options (dict): Dictionary of expansion options indexed by integers.
    """
    print(f"Using row {row_idx}: {row}")
    grammar = extract_grammar_from_data_row(row, columns)
    
    if current is None:
        current = "Z"

    # Get all possible expansions for the current expression
    expansions = expand_all_X(current, grammar)
    if not expansions:
        print("No expansions available.")
        return current, {}

    # Build an options dictionary mapping indices to expansion expressions
    current_options = {i: exp for i, exp in enumerate(expansions)}
    
    #print(f"\nCurrent expression: {current}")
    #print(f"Options dict:\n  {current_options}")

    return current_options


In [7]:
def compute_target(correct, input_row, nn_first_prediction, tree_candidate):
    """
    Given a truth table row, the neural network's first prediction, and a 
    candidate tree formula (tree_candidate), use find_allowable_combinations 
    to choose the closest candidate target vector.
    """
    assignments = tuple(input_row)

    # Run find_allowable_combinations using the tree candidate.
    candidate_list, non_terminal_count = find_allowable_combinations(tree_candidate, correct, assignments)
    
    # Convert each candidate dictionary into a vector.
    candidate_vectors = [
        torch.tensor(
            [candidate.get(f"Z_{i}", 0) for i in range(non_terminal_count)], dtype=torch.float
        )
        for candidate in candidate_list
    ]
    if not candidate_vectors:
        print(f"No candidate vectors for tree_candidate: {tree_candidate}")
        return torch.tensor([])  # Or handle this appropriately

    candidate_tensor = torch.stack(candidate_vectors)
    
    # Reshape nn_first_prediction to be a batch of size 1.
    nn_pred = nn_first_prediction.view(1, -1)
    
    # Compute Euclidean distances.
    distances = torch.norm(candidate_tensor - nn_pred, dim=1)
    best_idx = torch.argmin(distances).item()
    best_candidate = candidate_vectors[best_idx]

    return best_candidate


class Net(nn.Module):
    def __init__(self, input_size, output_size):
        """
        input_size: number of truth table inputs (here, 4 for P, Q, R, S)
        output_size: target vector length determined by non-terminals (e.g. 2)
        """
        super(Net, self).__init__()
        self.ln1 = nn.Linear(input_size, 16)
        self.ln2 = nn.Linear(16, 16)
        self.ln3 = nn.Linear(16, output_size)

    def forward(self, x):
        # Scale inputs from [0,1] to [-1, 1]
        x = 2 * x - 1
        if not isinstance(x, torch.Tensor):
            x = torch.Tensor(x)
        x = F.relu(self.ln1(x))
        x = F.relu(self.ln2(x))
        x = self.ln3(x)
        return torch.sigmoid(x)  # outputs in [0,1]



def train_on_truth_table(nn_model, truth_table, tree_candidate, max_iterations_per_row=10000):
    """
    For each truth table row, train the network until its rounded output
    matches the target computed for the given tree_candidate.
    
    Args:
        nn_model: The neural network model.
        truth_table: Truth table as a numpy array.
        tree_candidate: A candidate tree structure to be used in compute_target.
        max_iterations_per_row: Maximum number of training iterations per truth table row.

    Returns:
        iterations_per_row: A list of iteration counts for each truth table row.
    """
    optimizer = torch.optim.Adam(nn_model.parameters())
    iterations_per_row = []
    
    for i, row in enumerate(truth_table):
        row_input = np.array([row], dtype=np.float32) 
        
        with torch.no_grad():
            initial_pred = nn_model(torch.tensor(row_input))
        
        # Squeeze the prediction to shape (output_size,)
        initial_pred_vector = initial_pred.squeeze(0)
        
        # Compute the target for this row using the candidate tree.
        correct = bitlist[i]
        target = compute_target(correct, row, initial_pred_vector, tree_candidate)

        if target.numel() == 0:
            print(f"Skipping tree candidate due to empty target on row {i+1}")
            return None  # Signal that training was not possible for this tree

        
        training_iter = 0
        while training_iter < max_iterations_per_row:
            pred = nn_model(torch.tensor(row_input))
            pred_vector = pred.squeeze(0)
            pred_binary = round_prediction(pred_vector)
            
            # Check if the rounded prediction equals the target.
            if torch.equal(pred_binary, target):
                #print(f"Row {i+1} correct after {training_iter} iterations: pred {pred_binary.numpy()} vs. target {target.numpy()}")
                break  # move to next row
            else:
                loss = F.binary_cross_entropy(pred, target.unsqueeze(0))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                training_iter += 1
                
        if training_iter == max_iterations_per_row:
            print(f"Row {i+1} did not converge in {max_iterations_per_row} iterations.")
        iterations_per_row.append(training_iter)
        
    return iterations_per_row


def evaluate_candidate_options(current_options,
                               truth_table,
                               final_formula,
                               max_iterations_per_row=10000):
    """
    Runs through all tree-candidates in `current_options` USING
    the SAME nn_model instance, and either:
      • returns the matching formula-string (if a no‑X candidate
        exactly equals final_formula), or
      • returns a dict of {idx: total_iterations} for all candidates.
    """
    candidate_iterations = {}

    for idx, tree_cand in current_options.items():
        cand_str = tree_to_formula(tree_cand)
        #print(f"\nOption {idx}: {cand_str}")

        # If there are no Z‑slots, either it’s fully expanded or useless
        if cand_str.count('Z') == 0:
            if cand_str == final_formula:
                print(f"Matched at option {idx}")
                return cand_str
            else:
                print(f"Option {idx} has no non‑terminals; skipping.")
                continue

        # First, figure out how many non‑terminals we need to fill
        first_assignment = tuple(truth_table[0])
        _, non_terminal_count = find_allowable_combinations(
            tree_cand,
            correct=1,
            assignments=first_assignment
        )

        # Guard #1: if that count is zero, skip entirely
        if non_terminal_count == 0:
            print(f"Option {idx} yields zero non‑terminals; skipping.")
            continue

        # Build a dummy target to size the network
        dummy_prediction = torch.zeros(non_terminal_count)
        dummy_target = compute_target(
            1, first_assignment, dummy_prediction, tree_cand
        )
        output_size = dummy_target.shape[0]

        # Guard #2: if compute_target returned an empty target, skip
        if output_size == 0:
            print(f"Option {idx} gave empty targets; skipping.")
            continue

        # Now it’s safe to instantiate the model
        nn_model = Net(input_size, output_size)

        # Train on the full truth table
        iters = train_on_truth_table(
            nn_model, truth_table, tree_cand,
            max_iterations_per_row
        )
        if iters is None:
            print(f"→ Option {idx} skipped due to unsolvable tree.")
            continue

        total = sum(iters)
        candidate_iterations[idx] = total
        print(f"→ Total iterations: {total}")

    return candidate_iterations



if __name__ == "__main__":
    truth_table = np.array([
        [1, 1, 1, 1],
        [1, 1, 1, 0],
        [1, 1, 0, 1],
        [1, 1, 0, 0],
        [1, 0, 1, 1],
        [1, 0, 1, 0],
        [1, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 1, 1],
        [0, 1, 1, 0],
        [0, 1, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 1, 1],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0]
    ])
    input_size = 4

    found_count = 0
    found_formulas = []

    for row_index in range(100):
        row = all_rows[row_index]
        final_formula = row[formula_idx]
        bitlist = binary_to_bitlist(row[category_idx], len(truth_table))

        print(f"\n=== Trial for row {row_index}: target = {final_formula!r} ===")
        current = None

        for depth in range(10):
            current_options = run_derivation_for_row(
                row_index, row, column_names, current
            )

            result = evaluate_candidate_options(
                current_options,
                truth_table,
                final_formula,
                max_iterations_per_row=10000
            )

            if isinstance(result, str):
                print(f"Found minimal formula for row {row_index}: {result!r}")
                found_count += 1
                found_formulas.append((row_index, result))
                break
            else:
                best_key = min(result, key=result.get)
                current = current_options[best_key]


    print(f"\n=== Summary over 100 first rows ===")
    print(f"Found minimal formula in {found_count} / {100} cases.")
    for idx, formula in found_formulas:
        print(f" • row {idx}: {formula!r}")



=== Trial for row 0: target = 'p' ===
Using row 0: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65280, 0, 'p')
→ Total iterations: 40
Matched at option 1
Found minimal formula for row 0: 'p'

=== Trial for row 1: target = 'q' ===
Using row 1: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61680, 0, 'q')
→ Total iterations: 72
Option 1 has no non‑terminals; skipping.
Matched at option 2
Found minimal formula for row 1: 'q'

=== Trial for row 2: target = 'r' ===
Using row 2: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52428, 0, 'r')
→ Total iterations: 147
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Matched at option 3
Found minimal formula for row 2: 'r'

=== Trial for row 3: target = 's' ===
Using row 3: (0, 0, 0, 0, 0, 0, 1, 0, 0, 43690, 0, 's')
→ Total iterations: 49
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Matched at option 4
Found minimal formula for row 3: 's'

=== Trial for row 4: target = 'NA(p,p)' ===
Using ro

→ Total iterations: 139
→ Total iterations: 20
No candidate vectors for tree_candidate: ('NA', 'q', 'Z')
Skipping tree candidate due to empty target on row 5
→ Option 2 skipped due to unsolvable tree.
No candidate vectors for tree_candidate: ('NA', 'r', 'Z')
Skipping tree candidate due to empty target on row 3
→ Option 3 skipped due to unsolvable tree.
→ Total iterations: 25
Using row 7: (0, 0, 0, 0, 0, 0, 1, 0, 0, 22015, 1, 'NA(p,s)')
→ Total iterations: 90
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Matched at option 4
Found minimal formula for row 7: 'NA(p,s)'

=== Trial for row 8: target = 'NA(q,q)' ===
Using row 8: (0, 0, 0, 0, 0, 0, 1, 0, 0, 3855, 1, 'NA(q,q)')
→ Total iterations: 61
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; skipping.
Using row 8: (0, 0, 0, 0, 0, 0, 1, 0, 0, 3855, 1, 'NA(q,q)

→ Total iterations: 176
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; skipping.
Using row 12: (0, 0, 0, 0, 0, 0, 1, 0, 0, 30583, 1, 'NA(r,s)')
→ Total iterations: 60
No candidate vectors for tree_candidate: ('NA', 'p', 'Z')
Skipping tree candidate due to empty target on row 9
→ Option 1 skipped due to unsolvable tree.
No candidate vectors for tree_candidate: ('NA', 'q', 'Z')
Skipping tree candidate due to empty target on row 5
→ Option 2 skipped due to unsolvable tree.
→ Total iterations: 54
→ Total iterations: 1
Using row 12: (0, 0, 0, 0, 0, 0, 1, 0, 0, 30583, 1, 'NA(r,s)')
→ Total iterations: 56
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; skipping.
Using row 12: (0, 0, 0, 0, 0, 0, 1, 0, 0, 30583, 1, 'NA(r,s)')
→ Total iterations: 48
No candidate vectors for tree_candida

→ Total iterations: 140
→ Total iterations: 87
→ Total iterations: 78
Using row 15: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61695, 2, 'NA(p,NA(p,q))')
→ Total iterations: 74
→ Total iterations: 201
→ Total iterations: 40
→ Total iterations: 72
→ Total iterations: 125

=== Trial for row 16: target = 'NA(p,NA(p,r))' ===
Using row 16: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52479, 2, 'NA(p,NA(p,r))')
→ Total iterations: 119
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; skipping.
Using row 16: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52479, 2, 'NA(p,NA(p,r))')
→ Total iterations: 87
→ Total iterations: 23
No candidate vectors for tree_candidate: ('NA', 'q', 'Z')
Skipping tree candidate due to empty target on row 7
→ Option 2 skipped due to unsolvable tree.
No candidate vectors for tree_candidate: ('NA', 'r', 'Z')
Skipping tree candidate due to empty target on row 3
→ Option 3 skipped due to unsolvable tree.
No candidate

→ Total iterations: 43
→ Total iterations: 37
→ Total iterations: 0
No candidate vectors for tree_candidate: ('NA', 'p', ('NA', ('NA', 'p', ('NA', 'q', 's')), 'Z'))
Skipping tree candidate due to empty target on row 2
→ Option 4 skipped due to unsolvable tree.
Using row 18: (0, 0, 0, 0, 0, 0, 1, 0, 0, 49407, 2, 'NA(p,NA(q,r))')
→ Total iterations: 10
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; skipping.
Using row 18: (0, 0, 0, 0, 0, 0, 1, 0, 0, 49407, 2, 'NA(p,NA(q,r))')
→ Total iterations: 0
→ Total iterations: 1
→ Total iterations: 0
→ Total iterations: 0
→ Total iterations: 46

=== Trial for row 19: target = 'NA(p,NA(q,s))' ===
Using row 19: (0, 0, 0, 0, 0, 0, 1, 0, 0, 41215, 2, 'NA(p,NA(q,s))')
→ Total iterations: 77
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; skipp

...

→ Total iterations: 52
→ Total iterations: 63
No candidate vectors for tree_candidate: ('NA', ('NA', 'q', 'q'), 'Z')
Skipping tree candidate due to empty target on row 10
→ Option 2 skipped due to unsolvable tree.
No candidate vectors for tree_candidate: ('NA', ('NA', 'q', 'r'), 'Z')
Skipping tree candidate due to empty target on row 10
→ Option 3 skipped due to unsolvable tree.
→ Total iterations: 44
Using row 89: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61600, 3, 'NA(NA(p,q),NA(q,s))')
→ Total iterations: 99
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; skipping.
Using row 89: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61600, 3, 'NA(NA(p,q),NA(q,s))')
→ Total iterations: 39
→ Total iterations: 30
→ Total iterations: 20
No candidate vectors for tree_candidate: ('NA', ('NA', 'q', 's'), ('NA', 'r', 'Z'))
Skipping tree candidate due to empty target on row 4
→ Option 3 skipped due to unsolvable tree.
No candid

→ Total iterations: 87
→ Total iterations: 105
→ Total iterations: 63
→ Total iterations: 156
→ Total iterations: 119
Using row 91: (0, 0, 0, 0, 0, 0, 1, 0, 0, 63624, 3, 'NA(NA(p,q),NA(r,s))')
→ Total iterations: 119
→ Total iterations: 45
No candidate vectors for tree_candidate: ('NA', ('NA', 'q', 'q'), 'Z')
Skipping tree candidate due to empty target on row 10
→ Option 2 skipped due to unsolvable tree.
No candidate vectors for tree_candidate: ('NA', ('NA', 'q', 'r'), 'Z')
Skipping tree candidate due to empty target on row 10
→ Option 3 skipped due to unsolvable tree.
No candidate vectors for tree_candidate: ('NA', ('NA', 'q', 's'), 'Z')
Skipping tree candidate due to empty target on row 11
→ Option 4 skipped due to unsolvable tree.
Using row 91: (0, 0, 0, 0, 0, 0, 1, 0, 0, 63624, 3, 'NA(NA(p,q),NA(r,s))')
→ Total iterations: 103
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; ski

→ Total iterations: 52
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; skipping.
Using row 93: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52224, 3, 'NA(NA(p,r),NA(p,r))')
→ Total iterations: 49
→ Total iterations: 17
No candidate vectors for tree_candidate: ('NA', ('NA', ('NA', 'p', ('NA', 'r', 'q')), 'p'), ('NA', 'q', 'Z'))
Skipping tree candidate due to empty target on row 5
→ Option 2 skipped due to unsolvable tree.
→ Total iterations: 10
No candidate vectors for tree_candidate: ('NA', ('NA', ('NA', 'p', ('NA', 'r', 'q')), 'p'), ('NA', 's', 'Z'))
Skipping tree candidate due to empty target on row 6
→ Option 4 skipped due to unsolvable tree.

=== Trial for row 94: target = 'NA(NA(p,r),NA(p,s))' ===
Using row 94: (0, 0, 0, 0, 0, 0, 1, 0, 0, 60928, 3, 'NA(NA(p,r),NA(p,s))')
→ Total iterations: 42
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non

→ Total iterations: 58
→ Total iterations: 137
Using row 96: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52416, 3, 'NA(NA(p,r),NA(q,r))')
→ Total iterations: 66
No candidate vectors for tree_candidate: ('NA', ('NA', 'q', 'p'), 'Z')
Skipping tree candidate due to empty target on row 3
→ Option 1 skipped due to unsolvable tree.
No candidate vectors for tree_candidate: ('NA', ('NA', 'q', 'q'), 'Z')
Skipping tree candidate due to empty target on row 3
→ Option 2 skipped due to unsolvable tree.
→ Total iterations: 31
No candidate vectors for tree_candidate: ('NA', ('NA', 'q', 's'), 'Z')
Skipping tree candidate due to empty target on row 3
→ Option 4 skipped due to unsolvable tree.
Using row 96: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52416, 3, 'NA(NA(p,r),NA(q,r))')
→ Total iterations: 95
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; skipping.
Using row 96: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52416, 3, 'NA(NA(p,r),NA(q,

→ Total iterations: 29
No candidate vectors for tree_candidate: ('NA', ('NA', 'r', 'q'), 'Z')
Skipping tree candidate due to empty target on row 10
→ Option 2 skipped due to unsolvable tree.
No candidate vectors for tree_candidate: ('NA', ('NA', 'r', 'r'), 'Z')
Skipping tree candidate due to empty target on row 10
→ Option 3 skipped due to unsolvable tree.
→ Total iterations: 37
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')
→ Total iterations: 29
Option 1 has no non‑terminals; skipping.
Option 2 has no non‑terminals; skipping.
Option 3 has no non‑terminals; skipping.
Option 4 has no non‑terminals; skipping.
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')
→ Total iterations: 21
No candidate vectors for tree_candidate: ('NA', ('NA', 'r', 'p'), ('NA', 'p', 'Z'))
Skipping tree candidate due to empty target on row 9
→ Option 1 skipped due to unsolvable tree.
No candidate vectors for tree_candidate: ('NA', ('NA', 'r', 'p'), ('NA', 'q', 